### Computational Genomics
# Run a local BLAST+ search in Jupyter Notebook #
## to compare similarities of the Sars-nCoV-2 spike protein gene sequence and the Sars-nCoV genome to local and public databases ###

### written by Dr. Kat Valero, May 2020
k.wollenberg-valero@hull.ac.uk, https://github.com/cybokat
A friendly nod to https://github.com/davelunt, whose tutorial I couldn't get to work.


### On the line below please double click then type your name

Student:

## Assessed submission:
 **you will need to complete this notebook** and submit a PDF online. You will need to use the browser (Chrome) File menu to Print As PDF at the *end* of this practical class. Please make sure that all your work is entered in this workbook. It is essential that you fill out the `Assignment` sections in order to receive a grade.

## Learning Outcomes
At the end of this class you should:
- Become more proficient in **running docker images**
- Become more proficient in **using the unix command line**
- Learn about the **BLAST algorithm** and how it can be deployed from the command line
- Understand the differences between **local and remote BLAST searches**
- **Interpret sequence similarity** of Sars-nCOV-2 relative to other virus taxa

## 0. Getting Started
In this practical class you will build on your skills from the **Introduction to Bioinformatics** class. You will use a docker image of Jupyter Notebook (although this tutorial should also work with Jupyter Notebooks accessed in other ways), to compare the sequence similarity of the SARS-nCOV-2 Spike protein gene and its entire genome to local and remote databases using the BLAST algorithm.  

**DON’T PANIC! YOU DO NOT NEED COMPUTATIONAL SKILLS TO CARRY OUT THIS ASSESSMENT. EVERYTHING YOU NEED TO KNOW YOU CAN LEARN IN THIS CLASS**


### Setting up the Jupyter Notebook via docker and cloning the github repository
On your computer, make sure `Docker desktop` is running by checking the "show hidden icons" menu at the bottom right of your toolbar
Open a command line terminal and make sure no docker images are running with `docker ps`. If there is still a running image, terminate it with `docker stop XXXXX` where XXX is the ID of the image as seen in the `docker ps` output.   
Check whether you still have the Jupyter Notebook docker image installed with `docker images`. If not, pull it with `docker pull jupyter/datascience-notebook`. But it should still be on your machine. 
Run the image with `docker run -it --rm --name ds -p 8888:8888 jupyter/datascience-notebook`.
Open a browser window and enter `localhost:8888`. In the Jupyter Hub start page, enter the `token` which you can copy from your terminal window. 

Now that you have started Jupyter Hub, clone this repository via the `clone or download` button. Open a terminal in Jupyter Notebook and enter `git clone https://github.com/cybokat/Betacoronavirus_localBLAST_jpynb.git`. The notebook as well as some files should appear in the main directory of Jupyter Hub. Open the Jupyter Notebook and follow the instructions below. 



### Setting up cell magic
Make sure that cell magic is active to run bash. 
If not, use the `terminal` to interface bash like so: 
````
$ pip install rpy2
$ %load_ext rpy2.ipython
`````

### NOT looking at gigantic files in their entirety
in this tutorial `head` and `tail` will come in handy! Increase your number of lines to about -100 to be able to view results. 


## The Sars-Cov2-Spike protein S gene and blast searching

You should have gained some basic knowledge about the genome of the Sars novel Coronavirus causing the COVID-19 pandemic from the introductory lecture. This coronavirus posesses an envelope with embedded Spike proteins, which aid in its ability to infect human cells. You can find more information about the properties of the Spike protein, including references to its function, here: https://www.ncbi.nlm.nih.gov/gene/43740568. The Spike protein contains some conserved regions (amino acids that are necessary for its function and thus mutate only slowly**), so it is currently a target for the development of vaccines. 

** reference: Yuan, M., Wu, N.C., Zhu, X., Lee, C.C.D., So, R.T., Lv, H., Mok, C.K. and Wilson, I.A., 2020. A highly conserved cryptic epitope in the receptor binding domains of SARS-CoV-2 and SARS-CoV. Science, 368(6491), pp.630-633.https://pubmed.ncbi.nlm.nih.gov/32245784/ 

### Quiz 1: 
What is the purpose of a BLAST search? How would you accomplish it? Google is encouraged but please no Copy/Paste.
What could be the benefits and disadvantages of performing BLAST search locally as opposed to using the web interface?
Add a Markdown cell below this one with your answers. 

## 1 We begin by installing the latest version of the BLAST software

Install the latest version of the blast software (check the https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST address to see whether the name of the last version in the ftp directory matches with this code, if not, update). 

We use `wget` to pull the file from its web address:

`$ wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.10.0+-x64-linux.tar.gz`

Since the file is compressed (you can see that on its .tar.gz extension), we need to decompress it before use.

`$ tar -zxvf ncbi-blast-2.10.0+-x64-linux.tar.gz`

Add the Blast location to your system path. However, we will just add everything into the same folder later so that you won't get into any trouble matching the location of the software with the location of the files (a common source of despair). 

`$ export PATH=$home/jovyan/ncbi-blast-2.10.0+/bin:$PATH`

Replicate this code in the cell below, and check whether the software got downloaded and unpacked successfully

## 2 Make a local blast database 

The makeblastdb application within the BLAST+ software produces BLAST databases from FASTA files. This way you can convert any FASTA file containing sequences to a database, which you can use to identify matches to a query sequence. Assigning a unique identifier to every sequence in the database allows you to retrieve the matching sequence by its identifier and allows you to associate every sequence with a taxonomic node (through the taxid or taxonomic identifyer of the sequence). The identifier should begin right after the “>” sign on the definition line and contain no spaces.

After unpacking we will now perform all operations within the directory `ncbi-blast-2.10.0+/bin/`. Go to the Jupyter hub page, select all files beginning qith Covid_ and `move` them to this directory. When you enter code in cells, make sure you use `%%bash` and navigate to this directory (you already know how to do this). 

The code to make the blast database is as follows: 
```
$ ./makeblastdb -in databasefile.fas -dbtype nucl -out database 
````
where `databasefile.fas` is the FASTA file containing the sequences we wish to use as a database, and `database` is the database we produce from it (the output of ./makeblastdb). You can give this any name. `nucl` in `dbtype` specifies that our search should be based on nuceleotide sequence, the alternative for protein databases is`prot`. 

Now make a database from the sequences contained in Covid_genbank.fas, which contains sequences of Betacoronavirus Spike-protein genes that are not from the Sars-nCoV-2 virus strain. 

Check whether the database was generated using the Unix commands you already know. If you need help, the main directory should contain a **pdf of basic unix commands that we learnt in `Introduction to Bioinformatics`**. 

There should now be several files with different extensions, with the name you gave your new database. 

## 3 Use the blast database you created, to search for similarities against a query sequence

Now that you created the blast database, you can see how similar any sequence(s) is to the sequences in the databases. 
For this purpose, as the name implies, BLAST performs “local” alignments. Most proteins are modular in nature, with one or more functional domains occurring within a protein. The same domains may also occur in proteins from different species. The BLAST algorithm is tuned to find these domains or shorter stretches of sequence similarity. The local alignment approach also means that an mRNA can be aligned with a piece of genomic DNA, as is frequently required in genome assembly and analysis. If instead BLAST started out by attempting to align two sequences over their entire lengths (known as a global alignment), fewer similarities would be detected, especially with respect to domains and motifs.

The most commonly used BLAST algorithms are: 
- The BLASTN nucleotide-nucleotide search looks for more distant sequences.
- BLASTP performs protein-protein sequence comparison, and its algorithm is the basis of many other types of BLAST searches such   as BLASTX and TBLASTN.
- BLASTX searches a nucleotide query against a protein database, translating the query on the fly.
- TBLASTN searches a protein query against a nucleotide database, translating the database on the fly.

Our query sequence is the full sequence of the Sars-nCoV-2 Spike protein gene (S-gene). Therefore, we need to use `blastn`. 
The code for performing a BLAST search with `blastn` is: 
````
./blastn -db database -query queryfile.fas -out outfile.txt
````
where `database` is the data base we just generated with `makeblastdb`, `queryfile.fas` is the file containing our sequence query, and `outfile.txt` will contain the search results. 

Enter the commands below to search whether the Covid_sgene.fas file containing the sequence of the Spike protein matches with the entries of the database: 

## Assessment 1

Use the commands you learnt, to have a look at the output file from the search. Caution, it could be very large! Use the commands you know (a modification of `ls`) to find out its size, then have a look at the search results. Can you use google to interpret the parameters of the results table? Note down which organisms have the best hits against the query sequence. Enter your answer in a new cell below this one. 

I also added another resource for Coronavirus sequences from nextstrain.org. The file is called `Covid_nextstrain.fas`. Have a look at their website - we'll get back to it later in this course. Can you repeat the BLAST search against a database generated from the Nextstrain sequences? What are the top hits against that database? 
Enter them in a new cell below this one. 

## 4 Now access a genbank database of Betacoronavirus sequences to search for similarities against the entire Sars-CoV-2 genome

You can do this either remotely by accessing NCBI's servers, or you can download a local copy of this database to your directory. While the file might be large, the advantage is that you can access it anytime, independently of whether you have an internet connection. As we are using Jupyter Hub, we also won't have to do this on our local machine. 

The NCBI database FTP directory, including the new Betacoronavirus database, can be accessed at this address: 
 `https://ftp.ncbi.nlm.nih.gov/blast/db/`  and the database is called `Betacoronavirus.tar.gz`.  

Use your skills with `wget` and `tar -zxvf` to download and unpack the database to the home directory. If you're unsure, go back and have a look how we did it with the BLAST software. 

To make sure that we don't run into problems with the filepaths, look at the unpacked database (several files with different extensions and Betacoronavirus in the name), and `move` them from the home directory of Jupyter Hub into `ncbi-blast-2.10.0+/bin`, they should be in the same directory as the Covid_ files now. 

## Assessment 2

Now, I would like you to compare the entire genome of Sars-nCoV-2 against the database of Betacoronaviruses compiled by NCBI.I deposited it for you within the file `Covid_refgenome.fas`.  
You don't need the `makeblastdb` command here as the database has been prebuilt for you - you can directly move on to the next step which is the `blastn` search.  

Also try out the alternative of accessing not the `Betacoronavirus` database on your computer, but the `ref_viruses_rep_genomes` (Brister, 2015) database at NCBI, which contains virus reference genomes. For this, you need to add a `-remote` flag at the end of your code. Don't forget to rename your output file.

Now, have a look at the output produced by the blast search. Enter the code in the two cells below, for `Betacoronavirus` and for `ref_viruses_rep_genomes`.  

## 5 Limit the species you search against

Chances are, at least some of your database searches against `Betacoronavirus` or the remote search against `ref_viruses_rep_genomes` -- only returned Sars-CoV-2, which isn't surprising as that was your query sequence. However we would like to know if you compare the entire genome of the virus against other Coronaviruses, which other viruses are most closely related to Sars-CoV-2?

In short, we need a way to exclude our search term!

In order to limit a BLAST search by taxonomy, one needs to provide the NCBI taxonomy ID(s) (taxid) for given organism(s). A taxid is simply a number that specifies a node in the taxonomic tree. For example, 9606 is the taxid for human, 9989 is the taxid for rodentia, and 2 is the taxid for all bacteria. Taxids are preferable to organism names as the latter can be ambiguous. For instance, bacteria is both a genus of insects as well as a superkingdom. BLAST only accepts taxids that are at or below the species level. Additionally, one may use the -negative_taxids and -negative_taxidlist options to exclude sequences by taxid

Let's exclude Sars-CoV-2 by using `-negative_taxids` in the blast search. The Taxon ID for the novel Coronavirus is 2697049.
````
./blastn -db database -query queryfile.fas -negative_taxids 2697409 -out outfile.txt
````
Repeat your analysis of the `Covid_refgenome.fas` against the `Betacoronavirus` database, excluding the search taxon. Have a look at the results file. 

## Assessment 3. 

Having looked at the BLAST results of the Spike protein search and the Taxon-limited BLAST results of the entire genome,compare which organisms are infected by Coronaviruses that are similar in sequence to Sars-nCoV-2. Read the paper: 
`Lam, T.T.Y., Shum, M.H.H., Zhu, H.C., Tong, Y.G., Ni, X.B., Liao, Y.S., Wei, W., Cheung, W.Y.M., Li, W.J., Li, L.F. and Leung, G.M., 2020. Identifying SARS-CoV-2 related coronaviruses in Malayan pangolins. Nature, pp.1-6.`
which is provided to you as a pdf, and write a short 500-word paragraph explaining your results in comparison with those obtained in the paper. 
Add a Markdown cell below. Don't forget to upload your completed Jupyter Notebook for credits. 

### Sources: 
1. BLAST+ Command Line Applications User Manual. Accessed May 2020 at: https://www.ncbi.nlm.nih.gov/books/NBK279690/
2. Brister, J.R., Ako-Adjei, D., Bao, Y. and Blinkova, O., 2015. NCBI viral genomes resource. Nucleic acids research, 43(D1), pp.D571-D577.
4. Lam, T.T.Y., Shum, M.H.H., Zhu, H.C., Tong, Y.G., Ni, X.B., Liao, Y.S., Wei, W., Cheung, W.Y.M., Li, W.J., Li, L.F. and Leung, G.M., 2020. Identifying SARS-CoV-2 related coronaviruses in Malayan pangolins. Nature, pp.1-6.
5. Madden, T., The BLAST sequence analysis tool. In: the NCBI Handbook [Internet]. 2nd edition. Accessed May 2020 at: https://www.ncbi.nlm.nih.gov/books/NBK153387/